# Escansión automática con Rantanplan

- [Instalación de la librería spaCy para procesamiento del lenguaje](#instalar-spacy)
- [Instalar Rantanplan](#instalar-rantanplan)
- [Aplicar Rantanplan para la escansión (sobre un solo poema)](#escansion-rantanplan)
  - [Extraer los patrones métricos a partir del resultado de Rantanplan](#extraer-escansion)
  - [Formatear los resultados](#formatear-resultados)
    - [Añadir el número del verso](#anadir-numero-verso)
    - [Expresar los patrones métricos con cifras](#patron-metrico-cifras)
    - [Añadir el texto de cada verso](#anadir-texto-verso)

Este notebook muestra cómo utilizar [Rantanplan](https://github.com/linhd-postdata/rantanplan) para la escansión automática en español.

<a name="instalar-spacy"></a>

## Instalación de la librería spaCy (procesamiento del lenguaje)

Primero debemos instalar la librería spaCy en su versión 2.2.4. Rantanplan usa los análisis gramaticales proporcionados por ésta dentro de su análisis de la tonicidad léxica de las sílabas, que precede a la silabificación métrica y escansión.

El comando se precede de signo de exclamación porque no es código Python, sino una instrucción de la línea de comandos o shell del sistema.

In [5]:
!pip install -U spacy==2.2.4

  Using cached spacy-2.2.4-cp39-cp39-linux_x86_64.whl
  Using cached blis-0.4.1-cp39-cp39-linux_x86_64.whl
  Using cached thinc-7.4.0-cp39-cp39-linux_x86_64.whl
  Using cached srsly-1.0.6-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (209 kB)
  Using cached catalogue-1.0.2-py2.py3-none-any.whl (16 kB)
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.5
    Uninstalling srsly-2.4.5:
      Successfully uninstalled srsly-2.4.5
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: blis
    Found existing installation: blis 0.7.9
    Uninstalling blis-0.7.9:
      Successfully uninstalled blis-0.7.9
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.6
    Uninstalling thinc-8.1.6:
      Successfully uninstalled thinc-8.1.6
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uni

Los recursos de spaCy para ciertos análisis son específicos a cada lengua. Debemos descargar un modelo (conjunto de recursos) para español. Este modelo debe ser compatible con la versión 2.2.4 de spaCy. La instrucción para descargarlo es la siguiente. Esta instrucción no es la que se usa para descargar el último modelo de spaCy para español (`!python -m spacy download es_core_news_md`), sino una instrución usada para instalar una versión específica del modelo.

In [8]:
!pip install -U https://github.com/explosion/spacy-models/releases/download/es_core_news_md-2.3.1/es_core_news_md-2.3.1.tar.gz

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 1.4 MB/s eta 0:00:00m eta 0:00:010:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 602.4 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.6 MB/s eta 0:00:00m eta 0:00:010:00:01
  Created wheel for es-core-news-md: filename=es_core_news_md-2.3.1-py3-none-any.whl size=47447384 sha256=c8f643f49a194636f89055d15882fa364594c893c838901e7bfcc5643d4086e6
  Stored in directory: /home/ruizfabo/.cache/pip/wheels/ac/9e/58/009b585ec547a0666c0e72e44c39f42a8660fdeccd02803652
Successfully built es-core-news-md
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
  Attempting uninstall: es-core-news-md
   

También hay que instalar el módulo `spacy_affixes` para español, utilizado en algunos análisis.

In [15]:
!python -m spacy_affixes download es

/home/ruizfabo/anaconda3/envs/py39deezy/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


<a name="instalar-rantanplan"></a>

## Instalar Rantanplan

Una vez que spaCy está listo, se puede instalar Rantanplan

In [3]:
!pip install rantanplan

  Using cached rantanplan-0.7.1.tar.gz (807 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for rantanplan: filename=rantanplan-0.7.1-py2.py3-none-any.whl size=42519 sha256=e91cf67187b97211fba72e7ac3cff6dded6bfad9c68825bd82ea53004521292c
  Stored in directory: /home/ruizfabo/.cache/pip/wheels/45/5a/b5/1cbd3e5fb214b0e5f88ed4f73ac82c2e4f86d6032ae3635314
Successfully built rantanplan


<a name="escansion-rantanplan"></a>

## Aplicar Rantanplan para la escansión

Una vez que Rantanplan está instalado lo podemos usar para la escansión. Damos primero un ejemplo de código completo para obtener y mostrar en pantalla los patrones métricos de cada verso de un poema.

Después se explica el código paso a paso así como varias maneras de aumentar la legibilidad de los resultados (p. ej. expresar los patrones métricos como cifras en vez de con el formato por defecto que usa signos *más* y *menos* para las sílabas métricas tónicas y átonas).

In [ ]:
import spacy
import spacy_affixes
import rantanplan
from rantanplan.core import get_scansion

texto_poema = """Rosas sangrantes sobre el mar desflora
el sol que dice adioses en la tarde,
riman las aguas su canción sonora,
bajo nubes de fuego el poniente arde.

Vibran las cañas al chocar del viento,
formando extraña y triste sinfonía,
y la palmera altiva en vaivén lento
es una glauca nota de armonía.

Una barca se aleja lentamente,
una estela de luz, un vago canto,
sombras que pasan sobre el quieto mar;

Y las horas se van pausadamente,
mientras vierte la luz su último encanto
en un intenso, pálido llamear."""

resultado = get_scansion(texto_poema)
for verso in resultado:
    escansion = verso["rhythm"]["stress"]
    print(escansion)


Primero debemos importar los módulos necesarios:

- `spacy`: herramienta de PLN
- `spacy_affixes`: módulo necesario para algunos análisis gramaticales
- `rantanplan`: la herramienta de escansión
- `get_scansion`: la función que efectúa la escansión, que se encuentra en el paquete `rantanplan.core` de `rantanplan`

In [27]:
import spacy
import spacy_affixes
import rantanplan
from rantanplan.core import get_scansion

Declaramos una variable y le asignamos como valor la cadena de texto que representa el poema. La variable se ha llamado aquí `texto_poema`, pero podría usarse cualquier otro nombre que respete las [reglas de nombrado de variables](https://es.py4e.com/html3/02-variables) en el lenguaje Python.

El valor de `texto_poema` es una cadena de texto (tipo `str` en Python). Se ha escrito entre triples comillas, porque las triples comillas (o triple apóstrofe) permite que la cadena de texto tenga saltos de línea. En el poema cada verso empieza en su propia línea, es decir, va precedido de un salto de línea. Con las triples comillas, Python entiende que la cadena de texto sigue aunque contenga un salto de línea. 

In [28]:
texto_poema = """Rosas sangrantes sobre el mar desflora
el sol que dice adioses en la tarde,
riman las aguas su canción sonora,
bajo nubes de fuego el poniente arde.

Vibran las cañas al chocar del viento,
formando extraña y triste sinfonía,
y la palmera altiva en vaivén lento
es una glauca nota de armonía.

Una barca se aleja lentamente,
una estela de luz, un vago canto,
sombras que pasan sobre el quieto mar;

Y las horas se van pausadamente,
mientras vierte la luz su último encanto
en un intenso, pálido llamear."""

Para obtener la escansión, debemos aplicar la función `get_scansion()`. Ésta toma como argumento una cadena de texto. Los argumentos son los valores sobre los cuales la función opera, que se le dan entre los paréntesis, separados por comas. En este caso tenemos ya una variable, llamada texto_poema, con la cadena de texto, con lo que se pasa esta variable como argumento a la función.

Se ha asignado el valor del resultado a una variable, aquí llamada `resultado`. De este modo podemos reutilizar este resultado de la escansión, a través de la variable. 

In [29]:
resultado = get_scansion(texto_poema)

Para ver el valor de una variable, podemos escribir el nombre de esta. Escribiendo `resultado` vemos lo que Rantanplan ha anotado para el poema. Se explica a continuación la información proporcionada por Rantanplan y cómo extaerla para análisis posteriores.

In [30]:
resultado

[{'tokens': [{'word': [{'syllable': 'Ro', 'is_stressed': True},
     {'syllable': 'sas', 'is_stressed': False, 'is_word_end': True}],
    'stress_position': -2},
   {'word': [{'syllable': 'san', 'is_stressed': False},
     {'syllable': 'gran', 'is_stressed': True},
     {'syllable': 'tes', 'is_stressed': False, 'is_word_end': True}],
    'stress_position': -2},
   {'word': [{'syllable': 'so', 'is_stressed': False},
     {'syllable': 'bre',
      'is_stressed': False,
      'has_synalepha': True,
      'is_word_end': True}],
    'stress_position': 0},
   {'word': [{'syllable': 'el', 'is_stressed': False, 'is_word_end': True}],
    'stress_position': 0},
   {'word': [{'syllable': 'mar', 'is_stressed': True, 'is_word_end': True}],
    'stress_position': -1},
   {'word': [{'syllable': 'des', 'is_stressed': False},
     {'syllable': 'flo', 'is_stressed': True},
     {'syllable': 'ra', 'is_stressed': False, 'is_word_end': True}],
    'stress_position': -2}],
  'phonological_groups': [{'sylla

El resultado de la escansión (asignado a `resultado`) contiene varios tipos de información. Vemos aquí los contenidos más importantes del resultado y más abajo cómo explotarlos. El resultado contiene una lista de valores (un valor por verso) con una estructura concreta. La estructura del valor para cada verso se conoce como diccionario en Python (tipo `dict`). Un diccionario está estructurado según pares de clave-valor. Describimos las claves que nos interesan más directamente:
- `tokens`: contiene a su vez una lista de diccionarios, cada uno de los cuales tienen una clave word. El valor de `word` es a su vez una lista de diccionarios, uno por sílaba. Dentro de éstos, el valor de la clave `syllable` nos da la secuencia ortográfica para cada sílaba léxica del poema. P. ej. en el verso 4, veremos que los valores de `syllable` dentro de cada diccionario `word` son (separados con un guion): *ba-jo, nu-bes, de, fue-go, el, po-nien-te, ar-de*.
- `phonological_groups`: Representa la silabificación métrica mediante una lista de diccionarios. La clave `syllable` de cada diccionario permite acceder a las sílabas métricas. Por ejemplo en el verso 4, los valores de estas claves `syllable` son (separados con guion): *ba-jo-nu-bes-de-fue-goel-po-nien-tear-de* (sinalefas destacadas en negrita)
- `rhythm`: Es también un diccionario. Nos interesan sus claves `stress` y `length`.
  - `stress`: El patron métrico, con un signo más (+) para sílabas métricas tónicas y un signo menos (-) para las átonas. En el verso 4 el patrón es `--+--+--++-`, con acentos en las sílabas 3 y 6, el acento obligatorio en la 10 y un acento antirrítmico en la 9. 
  - `length`: Número de sílabas métricas del verso (en este poema siempre 11)

<a name="extraer-escansion"></a>


### Extraer los patrones métricos a partir del resultado de Rantanplan

El código para mostrar en pantalla el patrón métrico de cada verso es el siguiente. El código se explica línea  a línea en detalle más abajo. 

In [31]:
for verso in resultado:
    escansion = verso["rhythm"]["stress"]
    print(escansion)

+--+---+-+-
-+-+-+---+-
+--+---+-+-
--+--+--++-
+--+---+-+-
-+-+-+---+-
---+-+--++-
++-+-+---+-
+-+--+-+-+-
+-+--+++-+-
+--+---+-+-
--+--+-+-+-
--+--++--+-
-+-+-+---+-


Ya que queremos utilizar los resultados de escansión, vamos a extraer los valores de `stress` que están dentro del diccionario `rhythm` de cada verso. Explicamos cada línea del código.

- `for verso in resultado:`

  - El código para extraer la escansión empieza con un bucle (conjunto de instrucciones que se repiten para cada elemento de una serie). El bucle se introduce con la instrucción `for` y usa una variable, en este caso `verso`, para recorrer la serie (contenida en `resultado`). `for verso in resultado` quiere decir "recorrer la serie `resultado` asignando el contenido de cada elemento de la serie a `verso`, uno por uno".
  - Como se ha visto, `resultado` contiene una lista de diccionarios, y cada uno de estos diccionarios da la información anotada por Rantanplan para cada verso. A medida que el bucle va recorriendo los elementos de `resultado`, el valor de `verso` corresponde a la información de cada verso, uno por uno.
  - Las instrucciones dentro del bucle van indentadas (esto viene impuesto por la sintaxis del lenguaje Python, donde se indentan los bloques de código como bucles)

- `escansion = verso["rhythm"]["stress"]`
  - Para acceder a los valores de un diccionario, se usa el nombre de la clave asociada a esos valores. En caso de diccionarios imbricados (contenidos en otros diccionarios), se mencionan las claves sucesivas que llevan al valor deseado. Las claves se dan entre corchetes y entre comillas dentro de estos. 
  - El patron métrico se ha asignado a una variable llamada `escansion`
  - Como la expresión se encuentra dentro del bucle, se asigna *para cada verso* el patrón métrico a la variable

- `print(escansion)`
  - La instrucción print toma como argumentos (entre paréntesis, separados por comas) los valores a mostrar en pantalla. `print(escansion)` muestra el contenido asignado a la variable `escansion`.
  - Como la expresión se encuentra dentro del bucle, se muestra *para cada verso* el valor de `escansion` en pantalla
  
El resultado esperado es el siguiente (veremos más abajo cómo mejorar su legibilidad):

```
+--+---+-+-
-+-+-+---+-
+--+---+-+-
--+--+--++-
+--+---+-+-
-+-+-+---+-
---+-+--++-
++-+-+---+-
+-+--+-+-+-
+-+--+++-+-
+--+---+-+-
--+--+-+-+-
--+--++--+-
-+-+-+---+-
```

<a name="formatear-resultados"></a>

### Formatear los resultados

<a name="anadir-numero-verso"></a>

#### Añadir el número del verso

La legibilidad del resultado se puede mejorar de varias formas. Podemos empezar por numerar los resultados de cada verso:
- La instrucción `enumerate` numera cada elemento de la serie, es decir aquí numera las respuestas de Rantanplan para cada verso.
- El número creado por `enumerate` se debe asignar a una variable, en este caso `indice`.
- El valor de `indice` para cada verso corresponde al número creado por `enumerate` para él.
- Como se empieza a contar por cero (no por 1), al mostrar el número en pantalla se suma 1, con la expresión `indice+1` que es el primer argumento de `print` en la línea final del bucle.

El resultado esperado es el siguiente:

```
1 +--+---+-+-
2 -+-+-+---+-
3 +--+---+-+-
4 --+--+--++-
5 +--+---+-+-
6 -+-+-+---+-
7 ---+-+--++-
8 ++-+-+---+-
9 +-+--+-+-+-
10 +-+--+++-+-
11 +--+---+-+-
12 --+--+-+-+-
13 --+--++--+-
14 -+-+-+---+-
```

In [32]:
for indice, verso in enumerate(resultado):
    escansion = verso["rhythm"]["stress"]
    print(indice+1, escansion)

1 +--+---+-+-
2 -+-+-+---+-
3 +--+---+-+-
4 --+--+--++-
5 +--+---+-+-
6 -+-+-+---+-
7 ---+-+--++-
8 ++-+-+---+-
9 +-+--+-+-+-
10 +-+--+++-+-
11 +--+---+-+-
12 --+--+-+-+-
13 --+--++--+-
14 -+-+-+---+-


<a name="patron-metrico-cifras"></a>

#### Expresar los patrones métricos con cifras

Se proporciona primero el código y después su explicación detallada.

In [35]:
for indice, verso in enumerate(resultado):
    escansion = verso["rhythm"]["stress"]
    escansion_cifras = " ".join([str(i+1) for i, sig in
                                 enumerate(escansion) if sig == "+"])
    print(f"{indice+1}\t{escansion_cifras}")

1	1 4 8 10
2	2 4 6 10
3	1 4 8 10
4	3 6 9 10
5	1 4 8 10
6	2 4 6 10
7	4 6 9 10
8	1 2 4 6 10
9	1 3 6 8 10
10	1 3 6 7 8 10
11	1 4 8 10
12	3 6 8 10
13	3 6 7 10
14	2 4 6 10


Otra forma de mejorar la salida consiste en representar los patrones con cifras en vez de con signos más y menos.

Para esto usamos un nuevo bucle similar al usado para recorrer los resultados de Rantanplan, pero ahora lo usaremos para recorrer la cadena de texto que expresa el patrón métrico para cada verso.

La expresión `[str(i+1) for i, sig in enumerate(escansion) if sig == "+"]` es de tipo "comprensión de lista". Recorre una lista (en este caso el resultado de `enumerate(escansion)`, realiza operaciones sobre sus elementos, devuelve otra lista como salida:
- `for i, sig in enumerate(escansion)` recorre `escansion` con dos variables. La variable `i` corresponde al número creado por `enumerate` para cada signo dentro de `escansion`, y la variable `sig` corresponde a los signos.
- `enumerate(escansion)` se asigna un número a cada signo `+` y `-` del patrón.
- Gracias al test `sig == "+"` se consideran para la lista de salida sólamente las posiciones cuyo signo es "+"; el test sirve para filtrar la lista inicial y operar exclusivamente sobre las posiciones cuyo signo es `+`, que son las que tienen acento, cuya posición queremos reflejar con un número de sílaba en la salida.
- Como `enumerate()` empieza a numerar por cero (no por uno), debemos sumar 1 a sus números, esto se hace con `i+1`
- Finalmente, queremos una cadena de texto que exprese las posiciones acentuadas métricamente. Queremos texto, no números, con lo que convertimos con `str()` el resultado de `i+1`, con la expresión `str(i+1)`

Para obtener la cadena de texto final a partir del resultado de la comprensión de lista, usamos la expresión `join()`. Esta expresión une con un delimitador los elementos de una lista. Por ejemplo, para el primer verso, la comprensión de lista que convierte el patrón `+--+---+-+-` en números de posición (más exactamente en una cadena de texto que expresa cada número) devuelve la siguiente lista: `["1", "4", "8", "10"]`. Pero queremos la cadena `"1 4 8 10"`. La podemos obtener con `" ".join()`, que unirá con un espacio los elementos de la lista. La razón por la que hemos convertido a cadena cada número de la lista es que `join()` sólo opera sobre texto, no sobre números (no podemos darle la lista `[1, 4, 8, 10]` sino que hay que convertir cada número a cadena con `str()`, lo que da `["1", "4", "8", "10"]` como se ha mencionado).

Esta cadena de texto que proviene de aplicar `join()` sobre la lista devuelta por la comprensión de lista se ha asignado a la variable `escansion_cifras`.

Finalmente, para mostrar en pantalla los resultados se ha separado el número de verso del patrón métrico con una tabulación `\t`, para que se distinga mejor el número del verso del primer número de sílaba del patrón métrico. Cabe describir también el uso de la sintaxis `f""`, con una `f` antes de la cadena, para interpolar (intercalar) nombres de variable e incluso expresiones (porciones de código que devuelven un resultado) dentro de una cadena de texto; los nombres de variable y las expresiones irán entre llaves. Por eso escribimos dentro del argumento de print tanto la expresión `indice+1` como la variable `escansion_cifras` entre llaves.

El resultado esperado es el sigiuente:

```
1	1 4 8 10
2	2 4 6 10
3	1 4 8 10
4	3 6 9 10
5	1 4 8 10
6	2 4 6 10
7	4 6 9 10
8	1 2 4 6 10
9	1 3 6 8 10
10	1 3 6 7 8 10
11	1 4 8 10
12	3 6 8 10
13	3 6 7 10
14	2 4 6 10
```

<a name="anadir-texto-verso"></a>

#### Añadir el texto de cada verso

Esto requiere una manipulación adicional: Vamos a obtener el número de cada verso del poema a partir de su texto original, pero deberemos eliminar las líneas vacías (que separan las estrofas).

Para esto usamos una vez más una comprensión de lista. Mediante el test `if verso != ""`, la comprensión guarda para la lista de salida sólamente las líneas que no están vacías, que no son iguales a una cadena vacía `""`.

Se ha asignado la lista de salida a la variable `texto_sin_blancos`.

In [9]:
texto_sin_blancos = [verso for verso in texto_poema.split("\n") if verso != ""]

In [10]:
texto_sin_blancos

['Rosas sangrantes sobre el mar desflora',
 'el sol que dice adioses en la tarde,',
 'riman las aguas su canción sonora,',
 'bajo nubes de fuego el poniente arde.',
 'Vibran las cañas al chocar del viento,',
 'formando extraña y triste sinfonía,',
 'y la palmera altiva en vaivén lento',
 'es una glauca nota de armonía.',
 'Una barca se aleja lentamente,',
 'una estela de luz, un vago canto,',
 'sombras que pasan sobre el quieto mar;',
 'Y las horas se van pausadamente,',
 'mientras vierte la luz su último encanto',
 'en un intenso, pálido llamear.']

El texto de cada verso se muestra en pantalla gracias a `texto_sin_blancos[indice]` en la cadena procesada por la expresión `print()` en la última línea del bucle. Esto funciona por la razón siguiente: `indice` asigna un número a los resultados de Rantanplan para cada verso, empezando por 0. En Python se accede a los elementos de una lista usando índices numéricos. El texto sin líneas vacías está expresado en la lista `texto_sin_blancos`, cuyos elementos continenen el texto de cada verso. Podemos entonces usar los índices generados por `enumerate()` para acceder al texto de cada verso, dentro de la lista `texto_sin_blancos`.

In [41]:
for indice, verso in enumerate(resultado):
    escansion = verso["rhythm"]["stress"]
    escansion_cifras = " ".join([str(i+1) for i, ltr in enumerate(escansion) if ltr == "+"])
    print(f"{indice+1}\t{escansion_cifras}\t{texto_sin_blancos[indice]}")

1	1 4 8 10	Rosas sangrantes sobre el mar desflora
2	2 4 6 10	el sol que dice adioses en la tarde,
3	1 4 8 10	riman las aguas su canción sonora,
4	3 6 9 10	bajo nubes de fuego el poniente arde.
5	1 4 8 10	Vibran las cañas al chocar del viento,
6	2 4 6 10	formando extraña y triste sinfonía,
7	4 6 9 10	y la palmera altiva en vaivén lento
8	1 2 4 6 10	es una glauca nota de armonía.
9	1 3 6 8 10	Una barca se aleja lentamente,
10	1 3 6 7 8 10	una estela de luz, un vago canto,
11	1 4 8 10	sombras que pasan sobre el quieto mar;
12	3 6 8 10	Y las horas se van pausadamente,
13	3 6 7 10	mientras vierte la luz su último encanto
14	2 4 6 10	en un intenso, pálido llamear.


say what comments are
also write out the dataframe and on the book text give the screencap when you open it on a spreadsheet

In [55]:
import pandas as pd
from pathlib import Path

import rantanplan
from rantanplan.core import get_scansion

dir_poemas = "/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t"
ruta_poemas = Path(dir_poemas)

# preparar listas para los datos
poemas = []
numeros_poema = []
incipits = []
indices = []
escansiones = []
escansiones_cifras = []

# recorrer poemas y obtener escansión
for numero_poema, ruta_poema in enumerate(ruta_poemas.iterdir()):
    # mostrar el nombre de archivo de cada poema en pantalla
    print(ruta_poema)
    # procesar cada poema
    with open(ruta_poema) as descriptor_poema:
        # filtrar las líneas vacías si las hay
        texto_sin_blancos_lista = [linea for linea in descriptor_poema
                                   if linea != ""]
        # añadir contenido de la lista con el texto de cada verso
        # a la lista general de datos
        poemas.extend(texto_sin_blancos_lista)
        # poner texto del poema como CADENA para dar a rantanplan
        texto_sin_blancos = "".join(texto_sin_blancos_lista)
        # escansion (mismo proceso que para un solo poema)
        resultados = get_scansion(texto_sin_blancos)
        for indice, verso in enumerate(resultados):
            escansion = verso["rhythm"]["stress"]
            escansion_cifras = " ".join([str(i+1) for i, sig in
                                         enumerate(escansion)
                                         if sig == "+"])
            # añadir escansión a lista general de datos
            escansiones.append(escansion)
            escansiones_cifras.append(escansion_cifras)
            # añadir número de cada verso, íncipit y
            # un número para el poema a lista general
            indices.append(indice+1)
            incipits.append(texto_sin_blancos_lista[0].strip())
            numeros_poema.append(numero_poema+1)
            

# crear dataframe con las listas generales de datos
datos = {"numPoema": numeros_poema,
        "numVerso": indices,
        "incipit": incipits,
        "verso": poemas,
        "escanSig": escansiones,
        "escanNum": escansiones_cifras}

df = pd.DataFrame(datos)

/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0090.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0094.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0093.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0091.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0096.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0095.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0089.txt
/home/ruizfabo/projects/o/dhvol/capitulo-ana-poe/disco004t/disco004t_0092.txt


In [56]:
df

,numPoema,numVerso,incipit,verso,escanSig,escanNum
0,1,1,Rosas sangrantes sobre el mar desflora,Rosas sangrantes sobre el mar desflora\n,+--+---+-+-,1 4 8 10
1,1,2,Rosas sangrantes sobre el mar desflora,"el sol que dice adioses en la tarde,\n",-+-+-+---+-,2 4 6 10
2,1,3,Rosas sangrantes sobre el mar desflora,"riman las aguas su canción sonora,\n",+--+---+-+-,1 4 8 10
3,1,4,Rosas sangrantes sobre el mar desflora,bajo nubes de fuego el poniente arde.\n,--+--+--++-,3 6 9 10
4,1,5,Rosas sangrantes sobre el mar desflora,"Vibran las cañas al chocar del viento,\n",+--+---+-+-,1 4 8 10
...,...,...,...,...,...,...
107,8,10,En el vaso sombrío de mis penas,Quedó un dulzor de mieles en mi boca\n,-+-+-+---+-,2 4 6 10
108,8,11,En el vaso sombrío de mis penas,y algo muriente se fundió en el viento.\n,+--+---+-+-,1 4 8 10
109,8,12,En el vaso sombrío de mis penas,¡Ya dejé de llorar!... En mi horizonte\n,+-+--+---+-,1 3 6 10
110,8,13,En el vaso sombrío de mis penas,vi dibujarse una esperanza loca\n,+--++--+-+-,1 4 5 8 10
